In [1]:
import torch
import pandas as pd
from pathlib import Path
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from random import sample
import os
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import math
import torch.nn as nn


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [3]:
model_path = 'embeding_models/embeding_v3_8_dim.pt'
model = torch.jit.load(model_path).encoder.cpu()

In [4]:
BASE_DATE = datetime.strptime("2025-01-03", '%Y-%m-%d').timestamp()
print(BASE_DATE)

1735858800.0


In [5]:
tracks_raw_data = pd.read_json(Path('../data_v2/tracks_artists.jsonl'), lines=True)

# processs ARTIST HASH

def postprocess_hash_to_list(x):
    str_x = str(x)
    if len(str_x) < 8:
        str_x = "0" * (8 - len(str_x)) + str_x
    return [int(x) for x in str_x]

tracks_raw_data["id_artist_hash"] = tracks_raw_data["id_artist_hash"].apply(postprocess_hash_to_list)



In [6]:
class TracksDataset(Dataset):
    def __init__(self, tracks_data: pd.DataFrame):
        self.data = tracks_data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        unpacked_data = self.data.iloc[idx].drop("id_track").explode().values.astype(np.float64)
        return torch.from_numpy(unpacked_data)

In [7]:
embedings_per_id = pd.DataFrame()
embedings_per_id["id_track"] = tracks_raw_data["id_track"]
embedings_per_id["embeding"] = [model(torch.Tensor(x)).detach().cpu().numpy() for x in TracksDataset(tracks_raw_data)]

In [8]:
test_intigers = [687, 852, 528, 562, 426, 1092, 171, 250, 223, 265, 981, 607, 738, 1020, 510, 899, 596, 1047, 826, 669, 923, 905, 1063, 139, 1031]

In [9]:
files_to_load = [file for file in os.listdir("../data_v2/sessions") if int(file.split(".")[0].split("_")[-1]) not in test_intigers]

raw_sessions_data = pd.concat([pd.read_json(Path(f'../data_v2/sessions/{file}'), lines=True) for file in files_to_load])
print(raw_sessions_data.head())

   session_id               timestamp  user_id                track_id  \
0       89426 2024-09-17 10:51:23.000      260  1xPec5BN0Zxv77zrWKq43S   
1       89427 2023-05-20 00:35:36.227      260  7s0lDK7y3XLmI7tcsRAbW0   
2       89427 2023-05-20 00:38:48.054      260  48lQegoLqGAzaRLnMwK0mO   
3       89427 2023-05-20 00:43:30.802      260  48lQegoLqGAzaRLnMwK0mO   
4       89427 2023-05-20 00:44:59.387      260  4usVYcPlxRgRet6YashdCJ   

  event_type  
0       play  
1       play  
2       play  
3       like  
4       play  


In [144]:
merged_sesions_data = raw_sessions_data.merge(embedings_per_id, left_on="track_id", right_on="id_track").drop("id_track", axis=1)
merged_sesions_data["timestamp"] = (pd.to_datetime(merged_sesions_data["timestamp"], format='mixed').apply(lambda x: x.timestamp())).div(BASE_DATE)
merged_sesions_data = pd.get_dummies(merged_sesions_data, columns=["event_type"], dtype = int)

merged_sesions_data = [pd.DataFrame(y).sort_values("timestamp") for _, y in merged_sesions_data.groupby('user_id', as_index=False) if len(y) > 1]

NUMBER_OF_INPUTS = 12
EMBEDING_SIZE = 8

In [147]:
print(len(merged_sesions_data))

975


In [148]:
class CustomUserSesionsDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        sessions = []
        for session in self.data[idx].drop("track_id", axis=1).drop("session_id", axis=1).drop("user_id", axis=1).values:
            unpacked_data = np.array([])
            for data in session:
                unpacked_data = np.append(unpacked_data, data)
            sessions.append(torch.tensor(unpacked_data))
        return sessions

In [149]:
user_sesions_dataset = CustomUserSesionsDataset(merged_sesions_data)

In [150]:
COMPARE_PERCENTAGE = 0.25

def pad_sessions_collate(batch, pad_value=0.0):
    input_sequences = [
    ]
    output_sequences = [
    ]

    for batch_element in batch:
        number_of_sequence_data = len(batch_element)
        input_sequences.append(torch.stack(batch_element[:number_of_sequence_data - 1]))
        output_sequences.append(torch.stack([sample(batch_element[i+1:], 1)[0][1:EMBEDING_SIZE + 1] for i in range(number_of_sequence_data-1)]))

    x_lens = [len(x) for x in input_sequences]
    y_lens = [len(y) for y in output_sequences]

    padded_input_sequence = pad_sequence(input_sequences, batch_first=True, padding_value=pad_value)
    padded_output_sequence = pad_sequence(output_sequences, batch_first=True, padding_value=pad_value)

    return padded_input_sequence, padded_output_sequence, x_lens, y_lens


In [173]:
train_loader = DataLoader(user_sesions_dataset, batch_size=32, shuffle=True, collate_fn=pad_sessions_collate, num_workers=0)

In [252]:
# class UsertPreferenceGenerator(torch.nn.Module):
#     def __init__(self):
#         super(UsertPreferenceGenerator, self).__init__()
#         self.num_layers = 1
#         self.hidden_size = 256
#         self.rnn = torch.nn.LSTM(input_size=NUMBER_OF_INPUTS, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
#         self.act2 = torch.nn.LeakyReLU()
#         self.act1 = torch.nn.Hardsigmoid()
#         self.fc1 = torch.nn.Linear(self.hidden_size, 128)
#         self.fc2 = torch.nn.Linear(128, EMBEDING_SIZE)

#     def init_hidden(self, batch_size):
#         hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
#         state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
#         return hidden, state

#     def forward(self, x, x_lens, hidden):

#         x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
#         packed_output, hidden = self.rnn(x_packed, hidden)
#         output, _ = pad_packed_sequence(packed_output, batch_first=True)
#         x = self.fc1(self.act1(output))
#         x = self.fc2(self.act2(x))

#         return x, hidden


class UsertPreferenceGenerator(torch.nn.Module):
    def __init__(self):
        super(UsertPreferenceGenerator, self).__init__()
        self.num_layers = 3
        self.hidden_size = 32
        self.rnn = torch.nn.LSTM(input_size=NUMBER_OF_INPUTS, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.act2 = torch.nn.ReLU()
        self.act1 = torch.nn.Hardsigmoid()
        self.fc1 = torch.nn.Linear(self.hidden_size, 256)
        self.fc2 = torch.nn.Linear(256, EMBEDING_SIZE)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, x_lens, hidden):

        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.rnn(x_packed, hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        x = self.fc1(self.act1(output))
        x = self.fc2(self.act2(x))

        return x, hidden


class UsertPreferenceGenerator(torch.nn.Module):
    def __init__(self):
        super(UsertPreferenceGenerator, self).__init__()
        self.num_layers = 1
        self.hidden_size = 16
        self.rnn = torch.nn.LSTM(input_size=NUMBER_OF_INPUTS, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc1 = torch.nn.Linear(self.hidden_size, EMBEDING_SIZE)

    def reset_hidden_state(self, batch_size, target_device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(target_device)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(target_device)
        dummy_input_seq = torch.zeros(batch_size, 1, NUMBER_OF_INPUTS).to(target_device)
        _, _ = self.rnn(dummy_input_seq, (hidden, state))

    def forward(self, x, x_lens):

        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.rnn(x_packed)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        x_lens = torch.tensor(x_lens)
        mask = torch.arange(output.size(1)).expand(len(x_lens), output.size(1)) < x_lens.unsqueeze(1)
        mask = mask.unsqueeze(-1).to(output.device)
        output = output * mask
        x = self.fc1(output)
        # x = self.fc2(self.act2(x))

        return x

In [244]:
model = UsertPreferenceGenerator().to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_fun = nn.L1Loss()

# torch.onnx.export(model,[] ,'loop.onnx', verbose=True)

UsertPreferenceGenerator(
  (rnn): LSTM(12, 16, batch_first=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
)


In [253]:
# Training loop
model = UsertPreferenceGenerator().to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_fun = nn.L1Loss()


model.train()
for epoch in range(3):
    loss_sum = 0
    for x, targets, x_lens, y_lens in train_loader:
        x = x.float().to(device)
        targets = targets.to(torch.float).to(device)

        # hidden, state = model.init_hidden(x.size(0))
        # hidden, state = hidden.to(device), state.to(device)
        model.reset_hidden_state(x.size(0), device)
        preds = model(x, x_lens)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
    print(f"Epoch: {epoch}, loss: {loss_sum:.3}")

UsertPreferenceGenerator(
  (rnn): LSTM(12, 16, batch_first=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
)
Epoch: 0, loss: 17.6
Epoch: 1, loss: 17.4
Epoch: 2, loss: 17.1


In [154]:
torch.save(model, 'recomendation_models/user_preference_model_v4_8_dim.pt')

In [ ]:
import torch.autograd.profiler as profiler

with profiler.profile(use_cuda=True) as prof:
    preds, _ = model(x, x_lens, (hidden, state))
print(prof.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             aten::_pad_packed_sequence        10.52%       1.156ms        32.01%       3.519ms       3.519ms     909.000us         8.04%       3.522ms       3.522ms             1  
                                             aten::sort        12.44%       1.368ms        27.00%       2.968ms       1.484ms       1.377ms        12.18%       2.978ms       1.489ms             2  
         

STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [ ]:
model([[0.993923, 4.7692013 ,  0.50016433,  1.8391552 , -4.93348   ,  1.0898505 ,
          1.9289817 , 10.784728  , -3.9150748 ,  5.587146  ,  7.239292  ,
          7.1324625 ,  2.55823   ,  4.176082  ,  8.504707  ,  5.0005746 ,
         -2.202329, 0, 1, 0]], [1], torch.zeros(1, 1, 32))

AttributeError: 'list' object has no attribute 'device'

In [255]:
i = 0
for data in merged_sesions_data:
    sessions = []
    i += 1
    if i < 3:
        continue
    for session in (
        data.drop("track_id", axis=1)
        .drop("session_id", axis=1)
        .drop("user_id", axis=1)
        .values
    ):
        unpacked_data = np.array([])
        for element in session:
            unpacked_data = np.append(unpacked_data, element)
        sessions.append(torch.tensor(unpacked_data))
    sessions = torch.stack(sessions)
    # print(data)
    model.reset_hidden_state(1, "cpu")
    print(model.cpu()(torch.stack([sessions]).float(), torch.tensor([len(sessions)]))[-1][-4:-1])
    print("____________________________")
    if i > 25:
        break

/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)
/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


tensor([[-0.0981,  0.0610,  0.0832, -0.3069,  0.1439, -0.1514,  0.1113,  0.1178],
        [-0.1232,  0.0957,  0.0696, -0.3405,  0.1395, -0.1245,  0.0622,  0.0995],
        [-0.1292,  0.1090,  0.0691, -0.3464,  0.1300, -0.1199,  0.0538,  0.0855]],
       grad_fn=<SliceBackward0>)
____________________________
tensor([[-0.1013,  0.0348,  0.0529, -0.3173,  0.0960, -0.0907,  0.2120,  0.1766],
        [-0.1158,  0.0514,  0.0610, -0.3450,  0.0674, -0.0882,  0.1883,  0.1678],
        [-0.1015,  0.0612,  0.0499, -0.3369,  0.0846, -0.1231,  0.1725,  0.1506]],
       grad_fn=<SliceBackward0>)
____________________________


/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


tensor([[-0.1721,  0.1390,  0.0583, -0.3960,  0.1070, -0.1525,  0.0182,  0.1222],
        [-0.1241,  0.0740,  0.0466, -0.3479,  0.0833, -0.1208,  0.1321,  0.1440],
        [-0.1201,  0.0776,  0.0311, -0.3300,  0.0879, -0.1411,  0.1495,  0.1490]],
       grad_fn=<SliceBackward0>)
____________________________


/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


...

/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


tensor([[-0.1197,  0.0866, -0.0014, -0.3541,  0.0404, -0.1867,  0.1762,  0.1475],
        [-0.1037,  0.0494,  0.0133, -0.3343,  0.0235, -0.2160,  0.1977,  0.1540],
        [-0.1229,  0.0829,  0.0209, -0.3649,  0.0654, -0.1528,  0.1518,  0.1451]],
       grad_fn=<SliceBackward0>)
____________________________


/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


tensor([[-0.2041,  0.2105,  0.0106, -0.4202,  0.0838, -0.1769,  0.0554,  0.1405],
        [-0.1602,  0.1656,  0.0218, -0.4155,  0.0862, -0.1580,  0.1025,  0.1455],
        [-0.1585,  0.1953,  0.0505, -0.4218,  0.1350, -0.1226,  0.0018,  0.0977]],
       grad_fn=<SliceBackward0>)
____________________________
tensor([[-0.1428,  0.0684,  0.0578, -0.3311,  0.0755, -0.1418,  0.1199,  0.1385],
        [-0.1568,  0.1273,  0.0577, -0.3850,  0.1029, -0.1413,  0.0369,  0.1107],
        [-0.1499,  0.0897,  0.0877, -0.3457,  0.1200, -0.1110,  0.0470,  0.1328]],
       grad_fn=<SliceBackward0>)
____________________________
tensor([[-0.0629,  0.0565,  0.0967, -0.3116,  0.0932, -0.1199,  0.1206,  0.0985],
        [-0.0642,  0.0691,  0.1123, -0.3109,  0.1280, -0.0800,  0.0657,  0.0669],
        [-0.0881,  0.0882,  0.1150, -0.3313,  0.1060, -0.0826,  0.0126,  0.0435]],
       grad_fn=<SliceBackward0>)
____________________________


/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)
/tmp/ipykernel_4610/423004094.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lens = torch.tensor(x_lens)


In [187]:
model_scripted = torch.jit.script(model.cpu())
model_scripted.save("recomendation_models/scripted_user_preference_model_v4_8_dim.pt")